In [1]:
%run StdPackages.ipynb

Load wheels and test databases:

In [2]:
fs = [f"{d['data']}\\test_size1000.gdx", f"{d['data']}\\baselinerun.gdx"] # files
ws = gams.GamsWorkspace() 
g2np = gams2numpy.Gams2Numpy(ws.system_directory)
dbs = {'gms1': ws.add_database_from_gdx(fs[0]), 'gms2': ws.add_database_from_gdx(fs[1]),
       'gpy1': Database.GpyDB(db=fs[0],**{'name': 'testdb1'}), 'gpy2': Database.GpyDB(db=fs[1],**{'name': 'testdb2'})}
db = dbs['gpy2']

In [3]:
os.chdir(d['py'])
import DBWheels_agg
os.chdir(d['curr'])

# DBWheels_agg.py

The wheels ```DBWheels_agg.py``` specifies operations on entire databases e.g. for updating all sets from what is used in variables/parameters and aggregating in various ways. The main methods are: ```update_sets```

### 1: ```update_sets```

```python
DBWheels_agg.update_sets(db, types = ('variable','parameter'), clean = True, ignore_alias = False, clean_alias = False)
```

Goes through the following steps:
1. ```If clean```: Remove all elements from *sets* except for the alias-handling sets ```alias_set, alias_map2```.
2. Read sets from types: Iterate through all symbols of types ```types``` (default variable/parameter) and record all set elements. 
    * Aliased sets (listed in ```alias_map2```) can be recorded in the database as its own symbol or not (depends on the setting). The property ```db.alias_notin_db``` returns the aliases that are in ```alias_map2``` but is not defined as a symbol in ```db```.
    * ```If ignore_alias```: When reading sets from variables/parameters we will not write to the aliased sets in ```db.alias_notin_db```.
3. ```If clean_aliases```: If true all aliased sets that are not used in the database is removed from ```alias_map2```. 
4. Read aliased sets: This method gets the union of all aliased sets and updates accordingly.
    * ```If ignore_alias```: Once the union of all elements are detected the aliased sets in ```db.alias_notin_db``` are not updated.
5. ```If clean```: Update subsets and mappings based on the domain elements in the cleaned up sets.

*Executing the following is equivalent to simply using ```update_sets(db)```*.

In [4]:
#defaults:
types = ('variable','parameter')
clean = True
ignore_alias = False
clean_alias = False

*1. Clean: Returns empty pandas indices*

In [5]:
DBWheels_agg.clean_sets(db)
db.get('k')

Index([], dtype='object', name='k')

*2. Read sets from types: The set elements used in variables/parameters are read to the pandas indices again*

In [6]:
DBWheels_agg.read_sets(db, types = types, ignore_alias = ignore_alias)
db.get('k')

Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25],
      dtype='object', name='k')

*3. Clean aliases: Relies on the 'active_aliases' method to indicate which aliases are used in the database. Recall that aliases might be used in models even if they are not used here; thus we default to ```False```*

In [7]:
DBWheels_agg.active_aliases(db,types)

[('k', 'i'), ('k', 'j'), ('EA', 'EAEA')]

*Using the ```clean_aliases``` option would remove the aliases such as ```(HA,HAHA)```:*

In [8]:
db.get('alias_')

MultiIndex([(    'k',          'i'),
            (    'k',          'j'),
            ('aggid', 'aggidaggid'),
            (   'EA',       'EAEA'),
            (   'HA',       'HAHA')],
           names=['alias_set', 'alias_map2'])

*4. Read aliased sets: The set 'k' is aliased with ('i','j'). If ```read_sets``` has read elements to 'i' or 'j', these elements are in 'k' as well. In our case this does not make a difference*

In [9]:
DBWheels_agg.read_aliased_sets(db, ignore_alias = ignore_alias)
db.get('k')

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25],
           dtype='int64', name='k')

*5. Update subsets and mappings:*

In [10]:
DBWheels_agg.update_subsets_from_sets(db)
DBWheels_agg.update_maps_from_sets(db)